In [1]:
import os
import re
import torch
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from collections import Counter
from nltk.tokenize import word_tokenize

### Function to save and load files

In [2]:
def save_file(name, obj):
    with open(name, 'wb') as f:
        pickle.dump(obj, f)

def load_file(name):
    return pickle.load(open(name, 'rb'))

## Data Processing

In [3]:
tokens_path = 'Output/tokens.pkl'
file_path = 'Input/complaints.csv'
col_name = 'Consumer complaint narrative'

In [6]:
data = pd.read_csv('../2. MultiClass classification/Input/complaints.csv')

(2326246, 18)

In [7]:
data.shape

(2326246, 18)

### Drop missing values

In [8]:
data.dropna(subset=[col_name], inplace=True)

In [9]:
data.shape

(809343, 18)

In [10]:
input_text = data[col_name]

In [12]:
input_text[1]

'I contacted Ally on Friday XX/XX/XXXX after falling behind on payments due to being out of work for a short period of time due to an illness. I chated with a representative after logging into my account regarding my opitions to ensure I protect my credit and bring my account current. \n\nShe advised me that before an extenstion could be done, I had to make a payment in the amount of {$270.00}. I reviewed my finances, as I am playing catch up on all my bills and made this payment on Monday XX/XX/XXXX. This rep advised me, once this payment posts to my account to contact Ally back for an extention or to have a payment deffered to the end of my loan. \n\nWith this in mind, I contacted Ally again today and chatted with XXXX. I explained all of the above and the information I was provided when I chatted with the rep last week. She asked several questions and advised me that a one or two month  extension/deffered payment could be done however partial payment is needed! WHAT? She advised me 

### Convert text to lower case

In [15]:
input_text = [i.lower() for i in input_text]
input_text[1]

'hello this complaint is against the three credit reporting companies. xxxx, trans union and xxxx. i noticed some discrepencies on my credit report so i put a credit freeze with xxxx.on xx/xx/2019. i then notified the three credit agencies previously stated with a writtent letter dated xx/xx/2019 requesting them to verifiy certain accounts showing on my report they were a bankruptcy and a bank account from xxxx xxxx xxxx. \nthe response from xxxx and xxxx  was that it was verified through their third partner xxxx. that can not be correct because i have a freeze on my xxxx xxxx account since xx/xx/xxxx. which no one can obtain my report for anything until i unfreeze it. \ni wrote xxxx and xxxx a second letter and mailed them on xx/xx/2019 telling them that they have lied when they said that they verified the two accounts in question using xxxx because i have a freeze on my account and when i called xxxx on xx/xx/2019 they stated that no one has requested or attempted access to my accoun

### Remove punctuation marks